<center><h2>ALTeGraD 2023<br>Lab Session 4: NLP Frameworks</h2> 07 / 11 / 2023<br> Dr. G. Shang, H. Abdine<br><br>


<b>Student name:</b> [fill me]

</center>

In this lab you will learn how to use Fairseq and HuggingFace transformers - The most used libraries by researchers and developers  and finetune language models - to finetune a pretrained French language model ($RoBERTa_{small}^{fr}$) on the sentiment analysis dataset CLS_Books where each review is labeled as positive or negative and finetune a variant of BLOOM on a question/answer dataset.

## <b>Preparing the environment and installing libraries, models and data for Part 1 and Part 2</b>

In this section, we will setup the environment on Google Colab (first cell), download the pretraind model (second cell) and the finetuning dataset (third cell). In case you are using your personal computer maket sure to:

1- Use Ubuntu (or any similar linux distribution) or MacOS. <b> P.S. In case you have Windows, please use Google Colab. </b>

2- <b>Use Anaconda</b> and create new environment if you already installed Fairseq since we will be using a slightly modified version of this library.

In [ ]:
!mkdir altegrad.lab3 && cd altegrad.lab3 && mkdir libs
%cd altegrad.lab3/libs
!git clone https://github.com/hadi-abdine/fairseq
!pip install git+https://github.com/hadi-abdine/fairseq
!git clone https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install tensorboardX
%load_ext tensorboard

In [ ]:
!cd .. && mkdir models
%cd ../models

!wget -c 'https://nuage.lix.polytechnique.fr/index.php/s/E4otcD7B9jm2AWx/download/RoBERTa_small_fr.zip' -O "model_fairseq.zip"
!unzip model_fairseq.zip
!rm model_fairseq.zip
!rm -rf __MACOSX/

!wget -c "https://nuage.lix.polytechnique.fr/index.php/s/yYQjg9XWekttG5j/download/RoBERTa_small_fr_HuggingFace.zip" -O "model_huggingface.zip"
!unzip model_huggingface.zip
!rm model_huggingface.zip
!rm -rf __MACOSX/

In [ ]:
!cd .. && mkdir data
%cd ../data
!wget -c "https://nuage.lix.polytechnique.fr/index.php/s/EBHqfR776oCE2Nj/download/cls.books.zip" -O "cls.books.zip"
!unzip cls.books.zip
!rm cls.books.zip
!rm -rf __MACOSX/
!mkdir cls.books-json
%cd ..

# <b>Part 1: Fairseq</b>

In the first part of this lab, you will finetune the given model on model on CLS_Books dataset using <b>Fairseq</b> by following these steps:<br>

 1- <b>Tokenize the reviews</b> (Train, Valid and Test) using trained sentencepiece tokenizer provided alongside the pretrained model.[using sentencepiece library and setting the parameter <b>out_type=str</b> in the encode function].<br>
 2- <b>Binarize the tokenized reviews and their labels</b> using the preprocess python script provided in Fairseq.<br>
 3- <b>Fintune the pretrained $RoBERTa_{small}^{fr}$ model</b> using the train python script provided in Fairseq.<br>

 Finally, you will finish the first part by training a random $RoBERTa_{small}^{fr}$ model on the CLS_Books dataset and compare the results against the pretrained model while <b>visualizing the accuracies on tensorboard</b>.

## <b> Number of parameters of the model</b>

In this section you have to compute the number of parameters of $RoBERTa_{small}^{fr}$ using PyTorch (Only the base model with out the head). (<b>Hint:</b> you can check the architecture of the model using model['model'])

In [ ]:
import torch

model = torch.load("models/RoBERTa_small_fr/model.pt")

print("The total number of parameters: ",     ) #fill the gap

## <b>Tokenizing the reviews</b>

In this section we will tokenize the finetuning dataset using sentenpiece tokenizer. We have three splits in our datase: train valid and test sets.

In this task you have to use the trained sentencepiece tokenizer (RoBERTa_small_fr/sentencepiece.bpe.model) to tokenize the three files <b>train.review</b>, <b>valid.review</b> and <b>test.review</b> and output the three files <b>train.spm.review</b>, <b>valid.spm.review</b> and <b>test.spm.review</b> containing the tokenized reviews.

Documentation: https://github.com/google/sentencepiece#readme

In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='models/RoBERTa_small_fr/sentencepiece.bpe.model')

SPLITS=['train', 'test', 'valid']
SENTS="review"

for split in SPLITS:
    with open('data/cls.books/'+split+'.'+SENTS, 'r') as f:
        reviews = f.readlines()
        reviews = # tokenize the data using s.encode and a loop(check the documentation)

        # It should look something like this :
        #▁An ci enne ▁VS ▁Nouvelle ▁version ▁plus

    with open('data/cls.books/'+split+'.spm.'+SENTS, 'w') as f:
        for review in reviews:
          f.write(review+'\n')

## <b>Binarizing/Preprocessing the finetuning dataset</b>

In this section, you have to binarize the CLS_Books dataset using the <b>fairseq/fairseq_cli/preprocess.py</b> script:

1- Binarize the tokenized reviews and put the output in <b>data/cls-books-bin/input0</b>. Note: Our pretrained model's embedding matrix contains only the embedding of the vocab listed in the dictionary <b>dict.txt</b>. You need to use the dictionary in the binarization of the text to transform the tokens into indices. Also note that we are using Encoder only architecture, so we only have source data.

2- Binarize the labels (train.label, valid.label and test.label files) and put the output in <b>data/cls-books-bin/label</b>.

Documentation: https://fairseq.readthedocs.io/en/latest/command_line_tools.html

In [ ]:
!(python libs/fairseq/fairseq_cli/preprocess.py \
              --only-source \

              --workers 8)#fill me - binarize the tokenized reviews

!(python libs/fairseq/fairseq_cli/preprocess.py \
              --only-source \

              --workers 8)#fill me - binarize the labels

## <b>Finetuning $RoBERTa_{small}^{fr}$</b>

In this section you will use <b>fairseq/fairseq_cli/train.py</b> python script to finetune the pretrained model on the CLS_Books dataset (binarized data) for three different seeds: 0, 1 and 2.

Make sure to use the following hyper-parameters: $\textit{batch size}=8, \textit{max number of epochs}: 5, \textit{optimizer}: Adam, \textit{max learning rate}: 1e-05,  \textit{warm up ratio}: 0.06, \textit{learning rate scheduler}: linear$

In [ ]:
DATA_SET='books'
TASK= # fill me, sentence prediction task on fairseq
MODEL='RoBERTa_small_fr'
DATA_PATH= # fill me
MODEL_PATH= # fill me
MAX_EPOCH= # fill me
MAX_SENTENCES= # fill me, batch size
MAX_UPDATE= # fill me, number of backward propagation steps
LR= # fill me
VALID_SUBSET='valid,test' # for simplicity we will validate on both valid and test set, and then pick the value of test set corresponding the best validation score.
METRIC = # fill me, use the accuracy metric
NUM_CLASSES=#fill me, number of classes
SEEDS=3
CUDA_VISIBLE_DEVICES=0
WARMUP = # fill me, warmup ratio=6% of the whole training

In [ ]:
for SEED in range(SEEDS):
  TENSORBOARD_LOGS= 'tensorboard_logs/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/fairseq/fairseq_cli/train.py $DATA_PATH \
                --restore-file $MODEL_PATH \
                --batch-size $MAX_SENTENCES \
                --task $TASK \
                --update-freq 1 \
                --seed $SEED \
                --reset-optimizer --reset-dataloader --reset-meters \
                --init-token 0 \
                --separator-token 2 \
                --arch roberta_small \
                --criterion sentence_prediction \
                --num-classes $NUM_CLASSES \
                --weight-decay 0.01 \
                --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-08 \
                --maximize-best-checkpoint-metric \
                --best-checkpoint-metric 'accuracy' \
                --save-dir $SAVE_DIR \
                --lr-scheduler polynomial_decay \
                --lr $LR \
                --max-update $MAX_UPDATE \
                --total-num-update $MAX_UPDATE \
                --no-epoch-checkpoints \
                --no-last-checkpoints \
                --tensorboard-logdir $TENSORBOARD_LOGS \
                --log-interval 5 \
                --warmup-updates $WARMUP \
                --max-epoch $MAX_EPOCH \
                --keep-best-checkpoints 1 \
                --max-positions 256 \
                --valid-subset $VALID_SUBSET \
                --shorten-method 'truncate' \
                --no-save \
                --distributed-world-size 1)


## <b>Random $RoBERTa_{small}^{fr}$ model training:</b>

In this section you have to finetune a random checkpinf of the model $RoBERTa_{small}^{fr}$ using the same setting as before (<b>Hint:</b> an unexisted model path will not give you an error)

In [ ]:
DATA_SET='books'
TASK= # fill me, sentence prediction task on fairseq
MODEL='RoBERTa_small_fr_random'
DATA_PATH= # fill me
MODEL_PATH= # fill me
MAX_EPOCH= # fill me
MAX_SENTENCES= # fill me, batch size
MAX_UPDATE= # fill me, n_epochs * n_train_examples / total batch size
LR= # fill me
VALID_SUBSET='valid,test' # for simplicity we will validate on both valid and test set, and then pick the value of test set corresponding the best validation score.
METRIC = # fill me, use the accuracy metric
NUM_CLASSES=#fill me, number of classes
SEEDS=3
CUDA_VISIBLE_DEVICES=0
WARMUP = # fill me, warmup ratio=6% of the whole training

In [ ]:
for SEED in range(SEEDS):
  TENSORBOARD_LOGS= 'tensorboard_logs/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/fairseq/fairseq_cli/train.py $DATA_PATH \
                --restore-file $MODEL_PATH \
                --batch-size $MAX_SENTENCES \
                --task $TASK \
                --update-freq 1 \
                --seed $SEED \
                --reset-optimizer --reset-dataloader --reset-meters \
                --init-token 0 \
                --separator-token 2 \
                --arch roberta_small \
                --criterion sentence_prediction \
                --num-classes $NUM_CLASSES \
                --weight-decay 0.01 \
                --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-08 \
                --maximize-best-checkpoint-metric \
                --best-checkpoint-metric 'accuracy' \
                --save-dir $SAVE_DIR \
                --lr-scheduler polynomial_decay \
                --lr $LR \
                --max-update $MAX_UPDATE \
                --total-num-update $MAX_UPDATE \
                --no-epoch-checkpoints \
                --no-last-checkpoints \
                --tensorboard-logdir $TENSORBOARD_LOGS \
                --log-interval 5 \
                --warmup-updates $((6*$MAX_UPDATE/100)) \
                --max-epoch $MAX_EPOCH \
                --keep-best-checkpoints 1 \
                --max-positions 256 \
                --valid-subset $VALID_SUBSET \
                --shorten-method 'truncate' \
                --no-save \
                --distributed-world-size 1)

## <b>Tensorboard Visualisation </B>

In the this we will use tensorboard to visualize the training, validation and test accuracies. <b>Include and analyse in you report a screenshot of the test accuracy of the six models</b>.

In [ ]:
%tensorboard --logdir tensorboard_logs

# <b>Part 2: Finetuning $RoBERTa_{small}^{fr}$ using HuggingFace's Transfromers</b>
In this section of the lab, we will finetune a HuggingFace checkpoint of our $RoBERTa_{small}^{fr}$ on the CLS_Books dataset. Like in the first part we will start by downloading the HuggingFace checkpoint and <b>preparing a json format of the CLS_Books dataset</b> (Which is suitable for HuggingFace's checkpoints finetuning using their run_glue script).

## <b>Converting the CLS_Books dataset to json line files</b>

Unlike Fairseq, you do not need to perform tokenization and binarization in Hugging Face transformer library. However, in order to use the implemented script in the transformers library, you need to convert your data to json line files (for each split: train, valid and test)

for instance, each line inside you file will consist of one and one sample only, contaning the review (accessed by the key <i>sentence1</i> and its label, accessed by the key <i>label</i>. Below you can find an example from <i>valid.json</i> file.

Note that these instructions are not valid for all kind of tasks. For other types of tasks (supported in Hugging face) you have to refer to their github for more details.<br>

---------------------------------------------------------------------
<i>
{"sentence1":"Seul ouvrage fran\u00e7ais sur le th\u00e8me Produits Structur\u00e9s \/ fonds \u00e0 formule, il permet de fa\u00e7on p\u00e9dagogique d'appr\u00e9hender parfaitement les m\u00e9canismes financiers utilis\u00e9s. Une r\u00e9f\u00e9rence pour ceux qui veulent comprendre les technicit\u00e9s de base et les raisons de l'engouement des investisseurs sur ces actifs \u00e0 hauteur de plusieurs milliards d'euros.","label":"1"}<br>
{"sentence1":"Livre tr\u00e8s int\u00e9ressant !  mais si comme moi vous cherchez des \"infos\" sur les techniques de sorties et autres \"modes d'emploi\", afin de vivre par vous m\u00eame ce genre d'exp\u00e9rience, c'est pas le bon livre.  \u00e7a ne lui enl\u00e8ve d'ailleurd rien \u00e0 son int\u00earet.","label":"0"}
</i>

---------------------------------------------------------------------



In [ ]:
import json

SPLITS=['train', 'test', 'valid']

for split in SPLITS:
    with open('data/cls.books/'+split+'.review', 'r') as f:
        reviews = f.readlines()
    with open('data/cls.books/'+split+'.label', 'r') as f:
        labels = f.readlines()
    with open('data/cls.books-json/'+split+'.json', 'w') as f:
        #fill the gap here to create train.json, valid.json and test.json


## <b>Finetuning $RoBERTa_{small}^{fr}$ using the Transformers Library</b>

In order to finrtune the model using HuggingFace, you to use the <b>run_glue.py</b> Python script located in the transformers library. For more details, refer to <a href="https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification" target="_blank">the Huggingface/transformers repository on Github</a>. Make sure to use the same hyperparameter as in the first part of this lab.

In [ ]:
!pip install accelerate -U

In [ ]:
DATA_SET='books'
MODEL='RoBERTa_small_fr_huggingface'
MAX_SENTENCES= # fill me, batch size.
LR=#fill me, learning rate
MAX_EPOCH=#fill me
NUM_CLASSES=#fill me
SEEDS=3
CUDA_VISIBLE_DEVICES=0

In [ ]:
for SEED in range(SEEDS):
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/transformers/examples/pytorch/text-classification/run_glue.py \

  )#fill me

In [ ]:
%tensorboard --logdir checkpoints --port 6007

# <b>Part 2: Finetuning $BLOOM-560m$ using HuggingFace's Transfromers</b>
In this part, we will fintune $BLOOM-560m$:https://huggingface.co/bigscience/bloom-560m on a question/answer dataset. We will equally use LoRA and quantization during the finetuning.

## <b>Preparing the environment and installing libraries:<b>

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:</b>
In this section, we will load the BLOOM model while using the BitsAndBytes library for quantization.

In [ ]:
MODEL_NAME = "bigscience/bloom-560m"

bnb_config = # fill the gap

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        # fill the gap: get the number of trainable parameters: trainable_params
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## <b>Configuring LoRA:<b>

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

## <b>Test the model before finetuning:<b>

In [ ]:
prompt = # fill the gap, prompt of the format: "<human>: Comment je peux créer un compte?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## <b>Loading the question/answer dataset from HuggingFace:<b>

In [ ]:
data = load_dataset("OpenLLM-France/Tutoriel", data_files="ecommerce-faq-fr.json")
pd.DataFrame(data["train"])

## <b>Preparing the finetuning data:<b>

In [ ]:
def generate_prompt(data_point):
    return # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)

## <b>Finetuning:<b>

In [ ]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
def generate_response(question: str) -> str:
    prompt = # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [ ]:
prompt = "Puis-je retourner un produit s'il s'agit d'un article en liquidation ou en vente finale ?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Que se passe-t-il lorsque je retourne un article en déstockage ?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Comment puis-je savoir quand je recevrai ma commande ?"
print(generate_response(prompt))